<a href="https://colab.research.google.com/github/juboyy/Groq-Self-Learning/blob/main/groq-selfLearn-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install langchain_openai langchain-groq langchain_core python-dotenv langchain langchain_community

In [47]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

# **Vamos definir funções de apoio para cada habilidade do nosso agente**

In [36]:
# Gerenciamento de Histórico do chat (Chat History Mngmt.)

def get_chat_history(store, session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [37]:
# Gerar Resposta

def generate_response(chain_with_history, human_input: str, session_id: str, insights: str):
    response = chain_with_history.invoke(
        {"input": human_input, "insights": insights},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

In [38]:
# Reflexão
def reflect(llm, store, session_id: str):
    reflection_prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on the following conversation history, provide insights on how to improve responses:"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "Generate insights for improvement:")
    ])
    reflection_chain = reflection_prompt | llm
    history = get_chat_history(store, session_id)
    reflection_response = reflection_chain.invoke({"history": history.messages})
    return reflection_response.content

In [54]:
def learn(self, session_id: str):
    new_insights = self.reflect(session_id)
    learning_prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on these insights, update the agent's knowledge and behavior:"),
        ("human", "{insights}"),
        ("human", "Summarize the key points to remember:")
    ])
    learning_chain = learning_prompt | self.llm
    learned_points = learning_chain.invoke({"insights": new_insights}).content

    # Update the insights
    self.insights += f"\n{learned_points}"

    # Add the learned points to the chat history
    get_chat_history(self.store, session_id).add_ai_message(f"[SYSTEM] Agent learned: {learned_points}")

    return learned_points

In [55]:
# Self-Improving Agent Class
# vamos definir nossa classe de agente que utiliza todas estas funções acima

class SelfImprovingAgent:
    def __init__(self):
        self.llm = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", max_tokens=1000, temperature=0.7)
        self.store = {}
        self.insights = ""

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a self-improving AI assistant. Learn from your interactions and improve your performance over time. apenas responda em pt-br"),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}"),
            ("system", "Recent insights for improvement: {insights}")
        ])

        self.chain = self.prompt | self.llm
        self.chain_with_history = RunnableWithMessageHistory(
            self.chain,
            lambda session_id: get_chat_history(self.store, session_id),
            input_messages_key="input",
            history_messages_key="history"
        )

    def respond(self, human_input: str, session_id: str):
        return generate_response(self.chain_with_history, human_input, session_id, self.insights)

    def reflect(self, session_id: str):
        self.insights = reflect(self.llm, self.store, session_id)
        return self.insights

    def learn(self, session_id: str):
        self.reflect(session_id)

In [56]:
agent = SelfImprovingAgent()
session_id = "user_123"


# Interaction 1
print("AI:", agent.respond("What's the capital of Brazil?", session_id))

# Interaction 2
print("AI:", agent.respond("Can you tell me more about its history?", session_id))

# Learn and improve
print("\nReflecting and learning...")
learned = agent.learn(session_id)
print("Learned:", learned)

# Interaction 3 (potentially improved based on learning)
print("\nAI:", agent.respond("What's a famous landmark in this city?", session_id))

# Interaction 4 (to demonstrate continued improvement)
print("AI:", agent.respond("What's another interesting fact about this city?", session_id))


AI: Brasília
AI: Brasília é a capital do Brasil, localizada no Distrito Federal. Foi inaugurada em 21 de abril de 1960, como parte de um plano para transferir a capital do país de Rio de Janeiro para o interior. A cidade foi projetada pelos arquitetos Oscar Niemeyer e Lúcio Costa, e é conhecida por suas características arquitetônicas modernistas e urbanísticas.

Reflecting and learning...
Learned: None

AI: Um dos mais famosos marcos de Brasília é a Catedral Metropolitana, projetada por Oscar Niemeyer. A catedral é composta por 16 colunas de concreto que sustentam a estrutura, formando um espaço circular com 40 metros de diâmetro. Ela é um dos principais pontos turísticos da cidade e um exemplo da arquitetura modernista.
AI: Uma outra característica interessante de Brasília é que a cidade foi planejada de forma a ter um design de avião, com ruas projetadas como asas. Isso pode ser visto claramente de cima, e é um exemplo da inovadora urbanística da cidade.
